# Create WecOptTool Model for the Marson WEC concept

In [95]:
import logging
import numpy as np
import capytaine as cpt
import pygmsh
import gmsh
import vtk
import matplotlib.pyplot as plt
import os
import xarray as xr
from capytaine.io.meshio import load_from_meshio

logging.basicConfig(level=logging.INFO,format="%(levelnames)s:\t%(message)s")

#### WEC geometry mesh
Now we will create a surface mesh for the WEC hull and store it using the `FloatingBody` object from Capytaine.

In [96]:
numBodies = 2
bodyDist = 2
bodyLocs = [-0.5*bodyDist, -1.5*bodyDist] #, 3]
mesh_size_factor = 0.2
bodHeight = [1, 1]
bodDraft = [0.5, 0.5]
bodWidth = [1, 1]
bodLength = [1, 1]

hydrostaticsDict = {}
rect = {}
bodList = []

nBod = 0
with pygmsh.geo.Geometry() as geom:
    poly1 = geom.add_polygon(
        [[bodyLocs[nBod]-bodWidth[nBod]/2, bodLength[nBod]/2],
        [bodyLocs[nBod]-bodWidth[nBod]/2,-bodLength[nBod]/2],
        [bodyLocs[nBod]+bodWidth[nBod]/2,-bodLength[nBod]/2],
        [bodyLocs[nBod]+bodWidth[nBod]/2,bodLength[nBod]/2]],mesh_size=mesh_size_factor)
    
    geom.translate(poly1, [0, 0, bodHeight[nBod]-bodDraft[nBod]])
    geom.extrude(poly1,[0,0,-bodHeight[nBod]])
    rectMesh1 = geom.generate_mesh()

nBod = 1
with pygmsh.geo.Geometry() as geom:
    poly1 = geom.add_polygon(
        [[bodyLocs[nBod]-bodWidth[nBod]/2, bodLength[nBod]/2],
        [bodyLocs[nBod]-bodWidth[nBod]/2,-bodLength[nBod]/2],
        [bodyLocs[nBod]+bodWidth[nBod]/2,-bodLength[nBod]/2],
        [bodyLocs[nBod]+bodWidth[nBod]/2,bodLength[nBod]/2]],mesh_size=mesh_size_factor)
    
    geom.translate(poly1, [0, 0, bodHeight[nBod]-bodDraft[nBod]])
    geom.extrude(poly1,[0,0,-bodHeight[nBod]])
    rectMesh2 = geom.generate_mesh()

meshObj1 = load_from_meshio(rectMesh1, 'Rect1')
lidMesh1 = meshObj1.generate_lid(z=-.05)

# define the floating body
rect1 = cpt.FloatingBody(mesh=rectMesh1, lid_mesh = lidMesh1, name="rect_close", center_of_mass=(bodyLocs[0], 0, 0))
rect1.center_of_mass = (bodyLocs[0], 0, 0)
rect1.rotation_center = rect1.center_of_mass
#rect1.keep_immersed_part()
rect1.add_all_rigid_body_dofs()

meshObj2 = load_from_meshio(rectMesh2, 'Rect2')
lidMesh2 = meshObj2.generate_lid(z=-.05)

rect2 = cpt.FloatingBody(mesh=rectMesh2, lid_mesh = lidMesh2, name="rect_far", center_of_mass=(bodyLocs[1], 0, 0))
rect2.center_of_mass = (bodyLocs[1], 0, 0)
rect2.rotation_center = rect2.center_of_mass
#rect2.keep_immersed_part()
print(rect2.mesh.nb_faces)
rect2.add_all_rigid_body_dofs()

both_fb = cpt.FloatingBody.join_bodies(rect1,rect2)
both_fb.keep_immersed_part()

myBodies = []
myBodies.append(rect1)
myBodies.append(rect2)
print(rect2.mesh.nb_faces)

404
226


In [97]:
folder_path = 'hs'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for i, body in enumerate(bodList):

    cg = body.center_of_mass
    body_hs = body.compute_hydrostatics(rho=1023.0)
    vo = body_hs['disp_volume']
    cb = body_hs['center_of_buoyancy']
    khs = body_hs['hydrostatic_stiffness']

    # Write hydrostatic stiffness to KH.dat file
    khs_full = np.zeros((6,6))
    khs_full[2:5, 2:5] += khs[2:5, 2:5]
    tmp = folder_path + '/KH' + '_' + str(i) +'.dat'
    np.savetxt(tmp, khs_full)

    # Write the other hydrostatics data to Hydrostatics.dat file
    tmp = folder_path + '/Hydrostatics' + '_' + str(i) + '.dat'
    f = open(tmp,'w')
    for j in [0,1,2]:
        line =  f'XF = {cb[j]:7.3f} - XG = {cg[j]:7.3f} \n'
        f.write(line)
    line = f'Displacement = {vo:E}'
    f.write(line)
    f.close()


In [98]:
print(dir(rect1.mesh))
print(rect1.mesh.nb_faces)
print(rect2.mesh.nb_faces)
print(np.shape(rect2.dofs['Heave']))

['__abstractmethods__', '__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__internals__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rich_repr__', '__setattr__', '__short_str__', '__sizeof__', '__slotnames__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_clipping_data', '_compute_triangles_quadrangles', '_face_on_plane', '_faces', '_ids', '_repr_pretty_', '_vertices', 'as_set_of_faces', 'axis_aligned_bbox', 'center_of_buoyancy', 'center_of_mass_of_nodes', 'clip', 'clipped', 'compute_quadrature', 'copy', 'diameter_of_nodes', 'disp_mass', 'extract_faces', 'extract_lid', 'extract_one_face', 'faces', 'faces_areas', 'faces_centers', 'faces_normals', 'faces_radiuses', 'flip_normals', 'from_set_of_faces', 'generate_lid', 'get_face', 'heal_mesh', 'heal_normals',

In [99]:
from itertools import chain, accumulate, zip_longest
bodies = myBodies

"""Combine the degrees of freedom of several bodies."""
dofs = {}
cum_nb_faces = accumulate(chain([0], (body.mesh.nb_faces for body in bodies)))
total_nb_faces = sum(body.mesh.nb_faces for body in bodies)
for body, nbf in zip(bodies, cum_nb_faces):
    # nbf is the cumulative number of faces of the previous subbodies,
    # that is the offset of the indices of the faces of the current body.
    print('.......')
    print(body)
    print(body.mesh.nb_faces)
    for name, dof in body.dofs.items():
        new_dof = np.zeros((total_nb_faces, 3))
        print(name)
        print(np.shape(dof))
        
        print(nbf)
        print(nbf+len(dof))
        new_dof[nbf:nbf+len(dof), :] = dof
        if '__' not in name:
            new_dof_name = '__'.join([body.name, name])
        else:
            # The body is probably a combination of bodies already.
            # So for the associativity of the + operation,
            # it is better to keep the same name.
            new_dof_name = name
        dofs[new_dof_name] = new_dof

.......
FloatingBody(mesh=Mesh(..., name="mesh_from_meshio_500"), lid_mesh=Mesh(..., name="lid for Rect1"), dofs={"Surge": ..., "Sway": ..., "Heave": ..., "Roll": ..., "Pitch": ..., "Yaw": ...}, center_of_mass=(-1.0, 0, 0), name="rect_close")
223
Surge
(400, 3)
0
400
Sway
(400, 3)
0
400
Heave
(400, 3)
0
400
Roll
(400, 3)
0
400
Pitch
(400, 3)
0
400
Yaw
(400, 3)
0
400
.......
FloatingBody(mesh=Mesh(..., name="mesh_from_meshio_507"), lid_mesh=Mesh(..., name="lid for Rect2"), dofs={"Surge": ..., "Sway": ..., "Heave": ..., "Roll": ..., "Pitch": ..., "Yaw": ...}, center_of_mass=(-3.0, 0, 0), name="rect_far")
226
Surge
(404, 3)
223
627


ValueError: could not broadcast input array from shape (404,3) into shape (226,3)

In [100]:
ncFName = 'attenuator.nc'

path,tmp = os.path.split(os.getcwd() + os.path.sep + ncFName)
path += os.path.sep

# combine all bodies to account for B2B interactions
combo = myBodies[0]
combo += myBodies[1]

#both_fb = cpt.FloatingBody.join_bodies(rect1,rect2)

freqs = np.linspace(1/8, 10*1/8, 10)

# set frequencies
wCapy =  2*np.pi*freqs                  # wave frequencies
headings = np.linspace(0,np.pi/2,1)  
depth = np.infty
density = 1025.0

# call Capytaine solver
print(f'\n-------------------------------\n'
    f'Calling Capytaine BEM solver...\n'
    f'-------------------------------\n'
    f'mesh = {both_fb.name}\n'
    f'w range = {wCapy[0]:.3f} - {wCapy[-1]:.3f} rad/s\n'
    f'dw = {(wCapy[1]-wCapy[0]):.3f} rad/s\n'
    f'no of headings = {len(headings)}\n'
    f'no of radiation & diffraction problems = {len(wCapy)*(len(headings) + len(combo.dofs))}\n'
    f'-------------------------------\n')

problems = xr.Dataset(coords={
    'omega': wCapy,
    'wave_direction': headings,
    'radiating_dof': list(combo.dofs),
    'water_depth': [depth],
    'rho': [density],
    })

solver = cpt.BEMSolver()
capyData = solver.fill_dataset(problems, [combo], hydrostatics=False)

# save to .nc file
cpt.io.xarray.separate_complex_values(capyData).to_netcdf(ncFName, encoding={'radiating_dof': {'dtype': 'U'}, 'influenced_dof': {'dtype': 'U'}})
print('\nCapytaine call complete. Data saved to \n' + ncFName +'\n\n')

ValueError: could not broadcast input array from shape (404,3) into shape (226,3)